In [2]:
import pandas

In [1]:
import random

In [3]:
from sklearn.neighbors import KNeighborsClassifier

In [4]:
abalones = pandas.read_excel("dataset_processado.xlsx");
test = pandas.read_excel("app_processado.xlsx");

accuracy = {}; #Grupo de parâmetros: acurácia

In [5]:
#Parâmetros de teste
numReps = 5;
numChanges = 1;
randomSplit = 0.2;

numPop = 50;
mutationRate = 0.05;
generations = 100;
population = {}

for n in range(numPop):
    population[n] = [0,0,0,0,0,0,0,0,3]

In [6]:
def generatePop():
    for m in range(numPop):
        newPop = [];
        for n in range(8):
            newPop.append(random.randrange(101))
        newPop.append(random.randrange(3,100))
        population[m] = newPop;

In [7]:
def checkFitness(weights):
    accuracyResults = [];
    for num in range(numReps):
        knn = KNeighborsClassifier(n_neighbors = weights[8]);

        #Selecionando índices para teste e treino
        trainingIndexes = [];
        testIndexes = [];
        n = 0;
        for item in abalones["sex"]:
            if random.random() > randomSplit:
                trainingIndexes.append(n)
            else:
                testIndexes.append(n)
            n+=1

        #Criando os dataframes de treino e teste e separando os classificadores
        testData = abalones.drop(trainingIndexes);
        testClasses = testData["type"];
        testData = testData.drop(columns=["type"]);

        trainingData = abalones.drop(testIndexes);
        classifications = trainingData["type"]
        trainingData = trainingData.drop(columns=["type"])

        #Aplicando pesos
        n = 0;
        for col in trainingData.columns:
            trainingData[col] = trainingData[col]*weights[n]
            testData[col] = testData[col]*weights[n]
            n+=1

        #Aplicando K-NN
        knn.fit(trainingData, classifications)

        results = knn.predict(testData);

        #Testando acurácia
        n = 0;
        correctPredictions = 0;
        for item in testClasses:
            if item == results[n]:
                correctPredictions += 1;
            n+=1  

        acc = correctPredictions/len(results)
        accuracyResults.append(acc)

    return sum(accuracyResults)/len(accuracyResults)
    '''print("Acurácia média em " + str(numReps) + " tentativas: " + str(meanAcc))
    print("\n----- PARÂMETROS -----\nNúmero de Vizinhos: " + str(weights[8])
          + "\nSexo: " + str(weights[0]) 
          + "\nComprimento: " + str(weights[1])
          + "\nDiâmetro: " + str(weights[2])
          + "\nAltura: " + str(weights[3])
          + "\nPeso cheio: " + str(weights[4])
          + "\nPeso vazio: " + str(weights[5])
          + "\nPeso das vísceras: " + str(weights[6])
          + "\nPeso da concha: " + str(weights[7]) + "\n\n"
         )'''

In [8]:
def getFitness():
    results = {};
    for n in range(numPop):
        results[n] = checkFitness(population[n]);
        #print ("Checando a adequação de "+ str(n+1) + " de " + str(numPop) )
    return results

In [9]:
def selection(fitness):
    #Normalizando a chance de reprodução de acordo com o fitness
    best = max(fitness.values())
    worst =  min(fitness.values())
    for k in fitness.keys():
        fitness[k] = (fitness[k]-worst) / (best-worst)
    
    #Agora, com a tabela normalizada, temos a probabilidade de reprodução entre 0 e 1. Enquanto o tamanho da população não for
    #50% do original, continuamos removendo

    removed = 0;
    tries = 0;
    #print(numPop/2)
    while removed < (numPop/2) and tries < 2000:
        ind = random.randrange(numPop);
        #print("Índice: " + str(ind) + "\nParâmetros: " + str(population[ind]));
        #Ignorar se já foi removido
        if population[ind] == 0:
            tries+=1
            continue;
        else:
            if random.random() > fitness[ind]:
                #print("População removida.")
                removed+=1
                #print("Já removidos: " + str(removed))
                population[ind] = 0;
            else:
                tries+=1
    


In [15]:
def mutate(individual):
    n = 0
    for gene in individual:
        if random.random() < mutationRate:
            individual[n] = random.randrange(101)
        n+=1;
    if individual[-1] <= 2:
        individual[-1] = 3;
    return individual;

In [11]:
def crossover():
    newborns = 0;
    waitlist = []
    while newborns < numPop/2:
        #Selecionar aleatoriamente dois indivíduos
        ind1 = random.randrange(numPop);
        ind2 = random.randrange(numPop);
        
        while population[ind1] == 0:
            ind1 = random.randrange(numPop);
        while population[ind2] == 0 or ind2 == ind1:
            ind2 = random.randrange(numPop);
            
        #Indivíduos selecionados e conferidos, agora é necessário definir o ponto de cruzamento
        crosspoint = random.randrange(9)
        
        #Agora ocorre o real cruzamento. Ind1 doa a primeira parte e Ind2 doa a segunda parte.
        newInd = []
        newInd.extend(population[ind1][0:crosspoint])
        newInd.extend(population[ind2][crosspoint:])
        
        #Com o indivíduo novo criado, ele é adicionado a uma lista de espera (para não participar dos cruzamentos), após ser mutado
        newInd = mutate(newInd)
        waitlist.append(newInd)
        newborns+=1
    
    #Agora que acabaram os cruzamentos, precisamos repopular.
    for item in waitlist:
        if 0 in list(population.values()):
            population[list(population.values()).index(0)] = item
        

In [12]:
def geneticAlg():
    generatePop();
    gen = 0;
    while gen < generations:
        print("Geração: " + str(gen))
        fitness = getFitness()
        selection(fitness)
        crossover()
        gen+=1

In [16]:
geneticAlg()

Geração: 0
Geração: 1
Geração: 2
Geração: 3
Geração: 4
Geração: 5
Geração: 6
Geração: 7
Geração: 8
Geração: 9
Geração: 10
Geração: 11
Geração: 12
Geração: 13
Geração: 14
Geração: 15
Geração: 16
Geração: 17
Geração: 18
Geração: 19
Geração: 20
Geração: 21
Geração: 22
Geração: 23
Geração: 24
Geração: 25
Geração: 26
Geração: 27
Geração: 28
Geração: 29
Geração: 30
Geração: 31
Geração: 32
Geração: 33
Geração: 34
Geração: 35
Geração: 36
Geração: 37
Geração: 38
Geração: 39
Geração: 40
Geração: 41
Geração: 42
Geração: 43
Geração: 44
Geração: 45
Geração: 46
Geração: 47
Geração: 48
Geração: 49
Geração: 50
Geração: 51
Geração: 52
Geração: 53
Geração: 54
Geração: 55
Geração: 56
Geração: 57
Geração: 58
Geração: 59
Geração: 60
Geração: 61
Geração: 62
Geração: 63
Geração: 64
Geração: 65
Geração: 66
Geração: 67
Geração: 68
Geração: 69
Geração: 70
Geração: 71
Geração: 72
Geração: 73
Geração: 74
Geração: 75
Geração: 76
Geração: 77
Geração: 78
Geração: 79
Geração: 80
Geração: 81
Geração: 82
Geração: 83
Ge

In [17]:
population

{0: [72, 55, 21, 46, 85, 86, 27, 50, 68],
 1: [0, 66, 98, 54, 86, 86, 27, 50, 47],
 2: [0, 34, 64, 46, 85, 60, 27, 50, 68],
 3: [0, 34, 86, 46, 85, 97, 27, 35, 33],
 4: [0, 91, 58, 54, 86, 86, 27, 50, 68],
 5: [0, 34, 86, 46, 85, 60, 27, 50, 33],
 6: [0, 55, 21, 46, 85, 86, 27, 50, 33],
 7: [0, 56, 63, 77, 67, 86, 100, 50, 31],
 8: [0, 55, 86, 46, 85, 60, 27, 74, 68],
 9: [0, 34, 8, 46, 67, 60, 27, 50, 31],
 10: [0, 34, 64, 46, 85, 96, 27, 50, 31],
 11: [0, 34, 8, 46, 67, 60, 27, 50, 31],
 12: [0, 66, 98, 77, 67, 86, 27, 50, 31],
 13: [0, 34, 8, 46, 85, 60, 27, 49, 31],
 14: [0, 55, 21, 46, 86, 86, 66, 50, 31],
 15: [0, 55, 86, 46, 85, 60, 27, 50, 68],
 16: [0, 56, 63, 77, 63, 93, 27, 50, 33],
 17: [0, 66, 48, 46, 85, 86, 27, 50, 33],
 18: [0, 33, 98, 77, 85, 86, 27, 50, 33],
 19: [0, 55, 26, 86, 64, 86, 36, 49, 31],
 20: [72, 55, 21, 46, 85, 86, 27, 50, 33],
 21: [0, 34, 86, 46, 85, 60, 27, 50, 68],
 22: [0, 34, 86, 46, 85, 60, 45, 50, 68],
 23: [0, 56, 63, 77, 67, 86, 27, 50, 31],
 2

In [18]:
fitness = getFitness()

In [19]:
fitness

{0: 0.630248163680236,
 1: 0.658626330520155,
 2: 0.6628920910164464,
 3: 0.6696583113916217,
 4: 0.6404742968010955,
 5: 0.6543411959179899,
 6: 0.6444730873590524,
 7: 0.6391832235410924,
 8: 0.6437312904270802,
 9: 0.6477105465088937,
 10: 0.641493785726714,
 11: 0.6421429756830737,
 12: 0.6515270397381173,
 13: 0.6543766889104038,
 14: 0.634277570467458,
 15: 0.6376029946795179,
 16: 0.6518493119576163,
 17: 0.6413992079937872,
 18: 0.6543651036610564,
 19: 0.6506332077932359,
 20: 0.6413474194955036,
 21: 0.6337666430336253,
 22: 0.6507877267587444,
 23: 0.6714635547982499,
 24: 0.6473979299261726,
 25: 0.6489463388874771,
 26: 0.6455147923635097,
 27: 0.6576514856773549,
 28: 0.6523636927645409,
 29: 0.6378019413658391,
 30: 0.6547563893609095,
 31: 0.6440977828831002,
 32: 0.6356855068769021,
 33: 0.6450834339996184,
 34: 0.6336836710979086,
 35: 0.657584998472062,
 36: 0.640079381960279,
 37: 0.6578397239654329,
 38: 0.6577748067806753,
 39: 0.6630320049535171,
 40: 0.643333257

In [31]:
checkFitness([0, 56, 63, 77, 67, 86, 27, 50, 31])

0.6463903886088347

In [ ]:
#Geração Aleatória
for n in range(numChanges):
    weights = [random.random(),random.random(),random.random(),random.random(),random.random(),random.random(),random.random(),random.random(), random.randrange(3, 50, 2)]    
    accuracyResults = [];

    for n in range(numReps):
        knn = KNeighborsClassifier(n_neighbors = weights[8]);

        #Selecionando índices para teste e treino
        trainingIndexes = [];
        testIndexes = [];
        n = 0;
        for item in abalones["sex"]:
            if random.random() > randomSplit:
                trainingIndexes.append(n)
            else:
                testIndexes.append(n)
            n+=1

        #Criando os dataframes de treino e teste e separando os classificadores
        testData = abalones.drop(trainingIndexes);
        testClasses = testData["type"];
        testData = testData.drop(columns=["type"]);

        trainingData = abalones.drop(testIndexes);
        classifications = trainingData["type"]
        trainingData = trainingData.drop(columns=["type"])

        #Aplicando pesos
        n = 0;
        for col in trainingData.columns:
            trainingData[col] = trainingData[col]*weights[n]
            testData[col] = testData[col]*weights[n]
            n+=1

        #Aplicando K-NN
        knn.fit(trainingData, classifications)

        results = knn.predict(testData);

        #Testando acurácia
        n = 0;
        correctPredictions = 0;
        for item in testClasses:
            if item == results[n]:
                correctPredictions += 1;
            n+=1  

        acc = correctPredictions/len(results)
        accuracyResults.append(acc)

    meanAcc = sum(accuracyResults)/len(accuracyResults)
    print("Acurácia média em " + str(numReps) + " tentativas: " + str(meanAcc))
    print("\n----- PARÂMETROS -----\nNúmero de Vizinhos: " + str(weights[8])
          + "\nSexo: " + str(weights[0]) 
          + "\nComprimento: " + str(weights[1])
          + "\nDiâmetro: " + str(weights[2])
          + "\nAltura: " + str(weights[3])
          + "\nPeso cheio: " + str(weights[4])
          + "\nPeso vazio: " + str(weights[5])
          + "\nPeso das vísceras: " + str(weights[6])
          + "\nPeso da concha: " + str(weights[7])
         )
    accuracy[str(weights)] = meanAcc

In [ ]:
#Forward selection

def forwardSelection(baseWeights, baseAcc):
    m = 0;
    accs = [];
    for col in abalones.columns[:8]:
        weights = baseWeights.copy();
        weights[m] = 1
        accuracyResults = [];
    
        for num in range(numReps):
            knn = KNeighborsClassifier(n_neighbors = weights[8]);

            #Selecionando índices para teste e treino
            trainingIndexes = [];
            testIndexes = [];
            n = 0;
            for item in abalones["sex"]:
                if random.random() > randomSplit:
                    trainingIndexes.append(n)
                else:
                    testIndexes.append(n)
                n+=1

            #Criando os dataframes de treino e teste e separando os classificadores
            testData = abalones.drop(trainingIndexes);
            testClasses = testData["type"];
            testData = testData.drop(columns=["type"]);

            trainingData = abalones.drop(testIndexes);
            classifications = trainingData["type"]
            trainingData = trainingData.drop(columns=["type"])

            #Aplicando pesos
            n = 0;
            for col in trainingData.columns:
                trainingData[col] = trainingData[col]*weights[n]
                testData[col] = testData[col]*weights[n]
                n+=1

            #Aplicando K-NN
            knn.fit(trainingData, classifications)

            results = knn.predict(testData);

            #Testando acurácia
            n = 0;
            correctPredictions = 0;
            for item in testClasses:
                if item == results[n]:
                    correctPredictions += 1;
                n+=1  

            acc = correctPredictions/len(results)
            accuracyResults.append(acc)

        meanAcc = sum(accuracyResults)/len(accuracyResults)
        '''print("Acurácia média em " + str(numReps) + " tentativas: " + str(meanAcc))
        print("\n----- PARÂMETROS -----\nNúmero de Vizinhos: " + str(weights[8])
              + "\nSexo: " + str(weights[0]) 
              + "\nComprimento: " + str(weights[1])
              + "\nDiâmetro: " + str(weights[2])
              + "\nAltura: " + str(weights[3])
              + "\nPeso cheio: " + str(weights[4])
              + "\nPeso vazio: " + str(weights[5])
              + "\nPeso das vísceras: " + str(weights[6])
              + "\nPeso da concha: " + str(weights[7]) + "\n\n"
             )'''
        accs.append(meanAcc)
        m+=1
    
    #print("Acurácias finais: " + str(accs) + '\n\n')
    
    #Índice do melhor resultado
    ind = accs.index(max(accs))
    
    #Conferindo se houve melhora nessa recursão, caso não, já retorna a última lista de parâmetros
    if accs[ind] <= baseAcc:
        print("Resultado final: " + str(accs[ind]) + "%")
        return baseWeights
    #Caso haja melhora, a função será chamada novamente, com uma nova base de pesos e com a melhor acurácia
    else:
        newWeights = baseWeights.copy();
        newWeights[ind] = 1;
        return forwardSelection(newWeights, accs[ind])
    
    
for neighs in [5,15,25,50]:
    print(forwardSelection([0,0,0,0,0,0,0,0,neighs], 0))

#bestSet = forwardSelection([0,0,0,0,0,0,0,0,5], 0)